Task 2 - Recommender System

In [32]:
import pandas as pd
import numpy as np

Load the input file

In [33]:
file_folder = "Files"
file_name = "recom.csv"
path = f"{file_folder}//{file_name}"
df_recommender = pd.read_csv(path, header=0)
df_recommender.head()

,Unnamed: 0,Main_ID,Transaction_ID,Date,Price,Code_Product,Amount,ItemKey
0,0,90fada91,264f7a69,2022-10-07 20:53:49.153,125.0,5002.0,1.0,5002.0
1,1,9006f9ac,45c7d853,2022-09-17 15:54:57.187,19.0,35012.0,1.0,NaN
2,2,32270891,61ad76dd,2022-11-28 13:51:55.667,141.0,5005.0,1.0,5005.0
3,3,97e03e47,41ee09f6,2022-09-12 16:20:22.110,4.5,35078.5,1.0,NaN
4,4,41949228,244fe6d8,2022-10-14 18:53:43.933,129.5,49291.5,5.0,NaN


It looks like the Code_Product always equals the ItemKey, if the ItemKey as a value.
Need to check this in order to determine whether to remove the ItemKey column

In [34]:
def compare_Code_Product_with_ItemKey(df):
    print("Comparing the Code_Product column with the ItemKey column where the values are not null")
    df = df.dropna(subset="ItemKey")
    print(f"Comparing {len(df)} rows")
    result = df["Code_Product"].equals(df["ItemKey"])
    print(f"Both columns equal: {result}")

In [35]:
# df_recommender
compare_Code_Product_with_ItemKey(df_recommender)

Comparing the Code_Product column with the ItemKey column where the values are not null
Comparing 28597 rows
Both columns equal: True


Rename column names and convert data types. The Date column does not contain any useful information and can be removed. Drop the "ItemKey" column. Either it is the same as the "Code_Product" column or it is null 

In [36]:
rename_columns = {"Unnamed: 0" : "ID", "Main_ID": "Customer_ID", "Amount": "Order_Amount"}
df_recommender.rename(columns=rename_columns, inplace=True)
datatype_columns = {"Customer_ID" : "string", "Transaction_ID" : "string", "Order_Amount": int}
df_recommender = df_recommender.astype(datatype_columns)
df_recommender.drop(["Date", "ItemKey"], axis=1, inplace=True)
df_recommender.info()
df_recommender.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              50000 non-null  int64  
 1   Customer_ID     50000 non-null  string 
 2   Transaction_ID  50000 non-null  string 
 3   Price           50000 non-null  float64
 4   Code_Product    50000 non-null  float64
 5   Order_Amount    50000 non-null  int32  
dtypes: float64(2), int32(1), int64(1), string(2)
memory usage: 2.1 MB


,ID,Customer_ID,Transaction_ID,Price,Code_Product,Order_Amount
0,0,90fada91,264f7a69,125.0,5002.0,1
1,1,9006f9ac,45c7d853,19.0,35012.0,1
2,2,32270891,61ad76dd,141.0,5005.0,1
3,3,97e03e47,41ee09f6,4.5,35078.5,1
4,4,41949228,244fe6d8,129.5,49291.5,5


Pivot the dataframe for each customer

In [38]:
#pivot_columns = ["Code_Product", "Transaction_ID"]
#df_pivot = df_recommender.pivot(index="ID", columns="Customer_ID", values=pivot_columns )
#df_pivot.head()

Code_Product                                               \
Customer_ID     00024de6 00084856 0008e848 00096930 000c66b7 000e98ee   
ID                                                                      
0                    NaN      NaN      NaN      NaN      NaN      NaN   
1                    NaN      NaN      NaN      NaN      NaN      NaN   
2                    NaN      NaN      NaN      NaN      NaN      NaN   
3                    NaN      NaN      NaN      NaN      NaN      NaN   
4                    NaN      NaN      NaN      NaN      NaN      NaN   

                                                 ... Transaction_ID           \
Customer_ID 00165e2e 00171ffd 0019e439 001a93a4  ...       ffe473f3 ffe9b45e   
ID                                               ...                           
0                NaN      NaN      NaN      NaN  ...            NaN      NaN   
1                NaN      NaN      NaN      NaN  ...            NaN      NaN   
2                NaN      NaN      NaN      NaN  ...            NaN      NaN   
3                NaN      NaN      NaN      NaN  ...            NaN      NaN   
4                NaN      NaN      NaN      NaN  ...            NaN      NaN   

                                                                            \
Customer_ID ffea59de ffec4938 fff4483d fff8b1c4 fff905d0 fff9726b fffa332b   
ID                                                                           
0                NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1                NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2                NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3                NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4                NaN      NaN      NaN      NaN      NaN      NaN      NaN   

                      
Customer_ID ffff4aa9  
ID                    
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  

[5 rows x 57028 columns]